In [184]:
import pandas as pd
import numpy as np

In [ ]:
from random import randint

In [187]:
pd.DataFrame([
    ['Иван', 'Хлеб Бородинский', 1],
    ['Иван', 'Хлеб Белый', 0],
    ['Василий', 'Йогурт Epica', 1]
], columns=['user', 'item', 'purchase_fact'])

,user,item,purchase_fact
0,Иван,Хлеб Бородинский,1
1,Иван,Хлеб Белый,0
2,Василий,Йогурт Epica,1


In [277]:
recommended_list = [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43] #id товаров
bought_list = [521, 32, 143, 991]

In [ ]:
prices_recommended = [randint(100, 500) for i in range(10)]
prices_recommended

In [191]:
def hit_rate(recommended_list, bought_list):
    
    """был ли хотя бы 1 релевантный товар среди рекомендованных"""
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate = (flags.sum() > 0) * 1
    
    return hit_rate

In [206]:
def hit_rate_at_k(recommended_list, bought_list, k=5):
    
    """был ли хотя бы 1 релевантный товар среди топ-k рекомендованных"""
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    hit_rate_at_k = (flags.sum() > 0) * 1
    
    return hit_rate_at_k

In [205]:
def precision(recommended_list, bought_list):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    precision = flags.sum() / len(recommended_list)
    
    return precision

In [207]:
def precision_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:5]
          
    flags = np.isin(bought_list, recommended_list)
    
    precision_at_k = flags.sum() / len(recommended_list)
    
    return precision_at_k

In [273]:
def money_pricision_at_k(recommended_list, bought_list, prices_recommended, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    
    flags = np.isin(recommended_list, bought_list)
    
    money_pricision_at_k = (flags * prices_recommended).sum() / prices_recommended.sum()
    
    return money_pricision_at_k

In [271]:
def recall(recommended_list, bought_list):
    
    """доля рекомендованных товаров среди релевантных """
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(bought_list, recommended_list)
    
    recall = flags.sum() / len(bought_list)
    
    return recall

In [272]:
def recall_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    
    flags = np.isin(bought_list, recommended_list)
    
    recall_at_k = flags.sum() / len(bought_list)
    
    return recall_at_k

In [276]:
def money_recall_at_k(recommended_list, bought_list, prices_recommended, prices_bought, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)[:k]
    prices_recommended = np.array(prices_recommended)[:k]
    prices_bought = np.array(prices_bought)
        
    flags = np.isin(recommended_list, bought_list)
    
    money_recall_at_k = (flags * prices_recommended).sum() / prices_bought.sum()
    
    return money_recall_at_k

In [311]:
def ap_at_k(recommended_list, bought_list, k=5):
    
    bought_list = np.array(bought_list)
    recommended_list = np.array(recommended_list)
    
    flags = np.isin(recommended_list, bought_list)
    
    if not flags.sum(): return 0
    
    sum_ = np.array([
        precision_at_k(recommended_list, bought_list, k=i+1)
        for i in range(k-1) if flags[i]
    ]).sum()
    
    result = sum_ / flags.sum()
    
    return result

In [332]:
recommended_list = [
    [143, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43],
    [146, 156, 1134, 991, 27, 1543, 3345, 533, 11, 43]
] #id товаров
bought_list = [[521, 32, 143, 991], [146, 29]]

In [345]:
def map_at_k(recommended_list, bought_list, k=5):
    
    u = len(recommended_list)
    map_at_k = np.array(
        [ap_at_k(recommended_list[i], bought_list[i], k=k) 
        for i in range(u)]
    ).sum() / u
    
    return map_at_k

---

In [60]:
import torch

In [61]:
N = 5
ys_true = torch.randint(0, 5, (N, ))
ys_pred = torch.rand(N)

In [62]:
ys_true

tensor([1, 1, 3, 0, 4])

In [63]:
ys_pred

tensor([0.6729, 0.0711, 0.0695, 0.5213, 0.9205])

In [64]:
def compute_gain(y_value: float, gain_scheme: str) -> float:
    if gain_scheme == 'exp2':
        gain = 2 * y_value - 1
    elif gain_scheme == 'const':
        gain = y_value
    else:
        raise ValueError(F'{gain_scheme} метод не поддерживается, только exp2 и const')
    return float(gain)

In [65]:
def dcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str) -> float:
    _, argsort = torch.sort(ys_pred, descending=True, dim=0)
    ys_true_sorted = ys_true[argsort]
    ret = 0
    for idx, cor_y in enumerate(ys_true_sorted, 1):
        gain = compute_gain(cur_y, gain_scheme)
        ret += (2 ** gain - 1) / np.log2(1 + idx)
    return ret

In [66]:
def ndcg(ys_true: torch.Tensor, ys_pred: torch.Tensor, gain_scheme: str = 'const') -> float:
    
    pred_dcg = dcg(ys_true, ys_pred, gain_scheme)
    ideal_dcg = dcg(ys_true, ys_true, gain_scheme)
    
    ndcg = pred_dcg / ideal_dcg
    return ndcg

def ndcg(ys_true, ys_pred):
    def dcg(ys_true, ys_pred):
        _, argsort = torch.sort(ys_pred, descending=True, dim=0)
        ys_true_sorted = ys_true[argsort]
        ret = 0
        for i, l in enumerate(ys_true_sorted, 1):
            ret += (2 ** l - 1) / np.log2(1 + i)
        return ret
    ideal_dcg = dcg(ys_true, ys_true)
    pred_dcg = dcg(ys_true, ys_pred)
    return pred_dcg / ideal_dcg


In [68]:
 _, argsort = torch.sort(ys_pred, descending=True, dim=0)
print(ys_pred, argsort, sep='\n')

tensor([0.6729, 0.0711, 0.0695, 0.5213, 0.9205])
tensor([4, 0, 3, 1, 2])


In [55]:
ys_true

tensor([2, 4, 3, 1, 1])

In [54]:
ys_true_sorted = ys_true[argsort]
ys_true_sorted

tensor([1, 2, 3, 1, 4])

In [59]:
list(enumerate(ys_true_sorted, 1))

[(1, tensor(1)),
 (2, tensor(2)),
 (3, tensor(3)),
 (4, tensor(1)),
 (5, tensor(4))]